## Practico
### Introduccion al Aprendizaje Automatico

El objetivo consiste en explorar la aplicación de diferentes métodos de aprendizaje supervisado aprendidos 
en el curso, a través de experimentos reproducibles, y evaluando a su vez la conveniencia de uno u otro, 
así como la selección de diferentes hiperparámetros a partir del cálculo de las métricas pertinentes.

Para este practico se usara el dataset obtenido en el practico anterior
> Dataset Original: [here](https://drive.google.com/file/d/1wCNrWQbUd_bh1mdjZ-dg3g9y0r02JyQN/view?usp=sharing)

## 1. Preparación

* Usar la columna monto movimiento como target para modelos de regresión.
* Con el objetivo de preparar los datos para los modelos de clasificacion.
  Crear una columna que indique si un cliente ha aumentado el monto de movimiento 
  mesual, comparando con el mes anterior. En donde 
  - 1 si el monto ha aumentado respecto del mes anterior (con un incremento mayor o igual al 10%)
  - 0 si no fue asi.
  > consideraciones: el primer mes del dataset, el valor es 0
  
* Las variables explicativas (variables X's) no deben estar muy correlacionadas entre ellas, 
  ya que la variabilidad de una y otra estarán explicando la misma parte de variabilidad de la variable dependiente. 
  Esto es lo que se conoce como multicolinealidad, lo cual deriva en la imposibilidad de estimar los parámetros 
  cuando la misma es exacta o en estimaciones muy imprecisas cuando la misma es aproximada. En el caso de encontrar multicolinealidad, 
  - Cómo se puede solucionar? Qué decisión tomarían al respecto?
  
* Es posible que sea necesario normalizar las features de nuestro dataset. En qué casos tendrá que implementarse normalización?


## 2. División

La primer tarea consiste en dividir el conjunto de datos, en conjuntos de entrenamiento (training), validación (validation).
evaluación (testing) 

<img 
     src="https://i.stack.imgur.com/pXAfX.png" 
     alt="Drawing" 
     style="width:40%;"
/>

* Para el conjunto de evaluacion tomar el ultimo mes del dataset.
* Para entrenamiento y validación utilizar aproximadamente 70-80% de los datos para entrenamiento y 30-20% para validación.
* Es conveniente mezclar los datos aleatoriamente a la hora de realizar la división de los datos? 
 > `shuffle=True`?
* Pensar si hacer de esta forma la división puede afectar la distribución espacial y temporal de los datos. Cuáles pueden ser las consecuencias?

## 3. Regresión

* Se puede explicar el monto de los movimientos en base a la antiguedad del cliente?
  - Usar un modelo polinomial.
  - Cuál será la función de costo utilizada?
  - Grafique las curvas de error en términos del grado del polinomio. 
  - Seleccione el modelo que mejor funcione, y grafique el modelo conjuntamente con los puntos.

* Regresión con más de una variable
  - Explicar, Por qué se eligieron dichas variables?
  - Grafique las curvas de error en términos del grado del polinomio. 
  - Se obtuvieron mejores resultados? Por qué?
* Evaluar el modelo
  - Una vez elegido el mejor modelo. validar con el cojunto evaluación
  
 
> Agregar vector de Bias, cuando lo crean pertinente. Cuándo hace falta y cuándo no? Por qué?

## 4. Clasificación
* El TARGET se encuentra desbalanceado?
* Entrenar modelos lineales y No lineales de clasificación para predecir la variable objetivo.
* Seleccionar distintos valores para los hiperparámetros de los modelos usando `GridSearchCV`.
* Evaluar el modelo con el cojunto evaluación usando.
   - Accuracy
   - Precision
   - Recall
   - F1
   - matriz de confusión
* Explicar el por qué de cada resultado.

## 5. Optativo
* Graficar la curva ROC del modelo elegido en el punto anterior, Explicar.

Recuerden que la ciencia de datos es un proceso circular, continuo y no lineal. Es decir, si los datos requieren de mayor procesamiento para satisfacer las necesidades de algoritmos de ML (cualesquiera de ellos), vamos a volver a la etapa inicial para, por ejemplo, crear nuevas features, tomar decisiones diferentes sobre valores faltantes o valores atípicos (outliers), descartar features, entre otras.

In [43]:
import pyarrow
import fastparquet
import pandas as pd
import numpy as np

Para cargar el dataset

In [57]:
df = pd.read_parquet("df_final.parquet")

In [58]:
df

,dni_titular_movimiento,anio_mes_cupon,monto_ajustado,edad_cliente,antig_cliente,pca1,pca2,sexo_descripcion=Hombre,sexo_descripcion=Mujer,sexo_descripcion=Sin Datos,...,producto_naranja_movimiento=PP,producto_naranja_movimiento=TA,producto_naranja_movimiento=VI,producto_naranja_movimiento=ZE,tipo_producto_tarjeta_movimiento=0,tipo_producto_tarjeta_movimiento=22,tipo_producto_tarjeta_movimiento=3,tipo_producto_tarjeta_movimiento=32,tipo_producto_tarjeta_movimiento=4,tipo_producto_tarjeta_movimiento=42
0,000000b5aea2c9ea7cc155f6ebcef97f826,202008,-325.68,46.0,225,0.829899,-0.567124,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,000000b5aea2c9ea7cc155f6ebcef97f826,202009,-318.09,47.0,226,0.885993,-0.577546,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,000000b5aea2c9ea7cc155f6ebcef97f826,202010,-307.74,47.0,227,0.895144,-0.578143,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,000000b5aea2c9ea7cc155f6ebcef97f826,202011,-299.46,47.0,228,0.904295,-0.578743,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,000000b5aea2c9ea7cc155f6ebcef97f826,202012,-290.49,47.0,229,0.913446,-0.579343,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23203,000f0b73ebfa002a79a0642b82e87919904,202104,19956.87,64.0,24,-0.158297,-0.579022,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
23204,000f0b73ebfa002a79a0642b82e87919904,202104,61555.59,64.0,24,-0.146730,-0.495411,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
23205,000f0b73ebfa002a79a0642b82e87919904,202104,96119.07,64.0,24,-0.137119,-0.425941,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
23206,000f0b73ebfa002a79a0642b82e87919904,202104,145677.63,64.0,24,-0.123339,-0.326332,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [59]:
df.shape

(23208, 54)

Seleccionamos las columnas con mayor variabilidad en el tiempo en cada dni

## 1.1 Seleccion de columnas

In [60]:
# TODO: mejorar
columnas = [x for x in df.columns if 'dni_titular' in x 
                       or 'anio_mes' in x 
                       or 'comercio_cat' in x
                       or 'producto_naranja' in x
                       or 'tipo_producto' in x
                       or 'monto_ajustado' in x
                       or 'antig_cliente' in x
                       or 'pca' in x]

In [64]:
df_obj = df[columnas].copy()

## 1.2 Crear una columna que indique si un cliente ha aumentado el monto de movimiento mesual, comparando con el mes anterior

In [62]:
df_obj.shape

(23208, 32)

In [87]:
df_dni_mes = df_obj.groupby(['dni_titular_movimiento', 'anio_mes_cupon']).sum('monto_ajustado').reset_index()

In [93]:
df_anio_mes_min = df_obj.groupby(['dni_titular_movimiento'])[['dni_titular_movimiento', 'anio_mes_cupon']] \
    .min('anio_mes_cupon').reset_index() \
    .rename({'anio_mes_cupon':'anio_mes_min'}, axis=1)

In [97]:
df_dni_mes = df_dni_mes.merge(df_anio_mes_min, on='dni_titular_movimiento', how='left')

In [101]:
def aum_cons(row):
    if (row['anio_mes_cupon'] == row['anio_mes_min']):
        return 0
   # elif row['monto_ajustado'] > df_dni_mes[df_dni_mes['anio_mes_cupon'] == \
   #                                         row['anio_mes_cupon']-1]['monto_ajustado']*1.10:
 #       return 1
  #  else: 
  #      return 0

df_dni_mes['consumo_categorico'] = df_dni_mes.apply(aum_cons, axis=1)

In [107]:
df_dni_mes.groupby(['dni_titular_movimiento'])[['dni_titular_movimiento', 'anio_mes_cupon', 'consumo_categorico']] \
    .min('anio_mes_cupon').reset_index()['consumo_categorico'].unique()

array([0.])

In [44]:
target = np.array(df_obj.loc[:, 'monto_ajustado'])

In [45]:
target

array([  -325.68,   -318.09,   -307.74, ...,  96119.07, 145677.63,
       142402.89])